In [ ]:
import importlib
import classes.demography, classes.scale, classes.survey, classes.evaluation
importlib.reload(classes.demography)
importlib.reload(classes.survey)
importlib.reload(classes.scale)
importlib.reload(classes.evaluation)

In [ ]:
import pandas as pd
import statistics
from classes.survey import Survey, EvaluatedSurvey, AtWorkScenarios, CafeScenarios, OutsideScenarios, GrantedScenarios, HomeScenario, LocalNetworkCheck, LocalNetworkKnowledge, NoResponse, Answer, parse_survey
from classes.demography import Age, Gender, IT_Background, IOS_Version, Demography
from classes.scale import ATIScale, ScaleValue
from classes.evaluation import evaluate_answer, evaluate_surveys, get_ati_total_score, get_percentage_demographic, get_survey_time
from typing import List, Tuple, Dict
from collections import Counter
import csv
from scipy.stats import ttest_ind, chi2_contingency, fisher_exact, chisquare
from numpy import mean, array, var
from math import sqrt
import fisher


In [ ]:
survey_file = "../../data/dataset/user_study/local_network_survey.csv"
manual_ln_label_file = "../../data/dataset/user_study/manual_ln_label.csv"

In [ ]:
def parse_manual_file(file_path: str) -> Dict[str, EvaluatedSurvey]:
    # Parse the manual classified local network knowledge file
    surveys = {}
    c = 0
    with open(file_path, 'r') as f:
        reader = csv.reader(f, delimiter=';')
        for row in reader:
            if row[0] == 'id' or len(row[0]) == 0:
                continue
            survey = EvaluatedSurvey()
            survey.survey = Survey()
            survey.survey.prolific_id = row[0]
            if row[2] == "Good":
                survey.know_what_ln_is = True
                c = c + 1
            elif row[2] == "Bad":
                survey.know_what_ln_is = False
            else:
                survey.know_what_ln_is = True # WiFi
            
            surveys[row[0]] = survey
    return surveys


def update_local_network_knowledge_from_manual_analysis(evaluated_surveys: List[EvaluatedSurvey], manual_annotations: Dict[str, EvaluatedSurvey]):
    # Update the local network knowledge from the manual analysis
    for evaluated_survey in evaluated_surveys:
        if evaluated_survey.survey.prolific_id in manual_annotations:
            evaluated_survey.know_what_ln_is = manual_annotations[evaluated_survey.survey.prolific_id].know_what_ln_is


In [ ]:
# (and not if the answer was yes or no)

In [ ]:
#########################################
## Use Cases                            #
#########################################
# 
# Evaluate use cases from Q3.3
# 
# simply count peoples answers, i.e., how many true/false for each item separatly
# probably just report "don't know" separatly
# Report how many beleive in misconceptions or know it

def evaluate_use_cases(evaluated_surveys: List[EvaluatedSurvey]) -> Dict[str, int]:
    questions = {}
    for survey in evaluated_surveys:
        if not survey.sanety_check_passed:
            continue
        if HomeScenario.bluetooth not in survey.survey.at_home:
            questions["bluetooth_true"] = questions.get("bluetooth_true",  0) + 1
        elif HomeScenario.bluetooth in survey.survey.at_home:
            questions["bluetooth_false"] = questions.get("bluetooth_false", 0) + 1
        if HomeScenario.internet_access not in survey.survey.at_home:
            questions["internet_access_true"] = questions.get("internet_access_true", 0) + 1
        elif HomeScenario.internet_access in survey.survey.at_home:
            questions["internet_access_false"] = questions.get("internet_access_false", 0) + 1
        if HomeScenario.smart_cast in survey.survey.at_home:
            questions["smart_cast_true"] = questions.get("smart_cast_true", 0) + 1
        elif HomeScenario.smart_cast not in survey.survey.at_home:
            questions["smart_cast_false"] = questions.get("smart_cast_false",  0) + 1
        if HomeScenario.discover_other_phones in survey.survey.at_home:
            questions["discover_other_phones_true"] = questions.get("discover_other_phones_true", 0) + 1
        elif HomeScenario.discover_other_phones not in survey.survey.at_home:
            questions["discover_other_phones_false"] = questions.get("discover_other_phones_false",  0) + 1
        if HomeScenario.none_of_above not in survey.survey.at_home:
            questions["none_of_above_true"] = questions.get("none_of_above_true", 0) + 1
        elif HomeScenario.none_of_above in survey.survey.at_home:
            questions["none_of_above_false"] = questions.get("none_of_above_false", 0) + 1
        if HomeScenario.dont_know in survey.survey.at_home:
            questions["dont_know"] = questions.get("dont_know", 0) + 1
    return questions


In [ ]:
def count_answers(answers: List[Answer], answer: Answer) -> Dict[str, int]:
    result = {
        "count_true": 0,
        "count_false": 0,
        "count_dont_know": 0
    }
    wrong_answer = Answer.no if answer == Answer.yes else Answer.yes

    for a in answers:
        if evaluate_answer(a, answer):
            result["count_true"] = result.get("count_true", 0) + 1
        elif evaluate_answer(a, wrong_answer):
            result["count_false"] = result.get("count_false", 0) + 1
        elif evaluate_answer(a, Answer.dont_know):
            result["count_dont_know"] = result.get("count_dont_know", 0) + 1
        else:
            print("error neither true nor false nor don't know")
    return result

def count_permission_seen(evaluated_surveys: List[EvaluatedSurvey]) -> Dict[str, int]:
    answers = [survey.survey.encountered_permission for survey in evaluated_surveys if survey.sanety_check_passed]
    return count_answers(answers, Answer.yes)


def count_ln_knowledge(evaluated_surveys: List[EvaluatedSurvey]) -> Dict[str, int]:
    result = {
        "count_true": 0,
        "count_false": 0,
    }
    for survey in evaluated_surveys:
        if survey.sanety_check_passed:
            if survey.know_what_ln_is:
                result["count_true"] = result.get("count_true", 0) + 1
            else:
                result["count_false"] = result.get("count_false", 0) + 1
    return result

#########################################
## Attacker Models                      #
#########################################
# 1) Threat of exposing offline devices (Q60)
#
# QUESTION: What are we doing with the "don't know"? It could be that
# those people did not understand der question... When too many we need
# think how to handle this question
#
# Count awarness
def evaluate_exposing_devices(evaluated_surveys: List[EvaluatedSurvey]) -> Dict[str, int]:

    answers = [survey.survey.granted_scenarios.exposing_devices for survey in evaluated_surveys if survey.sanety_check_passed]

    return count_answers(answers, Answer.yes)


# 2) People co-locating (Q58)
#
# also here probably just count true/false answers, then we report
# how many are aware of this threat.
#
def evaluate_co_locating(evaluated_surveys: List[EvaluatedSurvey]) -> Dict[str, int]:

    answers = [survey.survey.granted_scenarios.cross_user_tracking for survey in evaluated_surveys if survey.sanety_check_passed]

    return count_answers(answers, Answer.yes)


# FIXME: what to do with don't know + false

# 3) Location Profiling (Q56, Q57)

# check how many people know about this threat (count true/false)
# score + if people get one of them, and none right
def evaluate_location_profiling(evaluated_surveys: List[EvaluatedSurvey]) -> Dict[str, int]:
    result = {
        "count_true": 0,
     #   "count_profiling": 0,
     #   "count_location": 0,
        "count_false": 0,
        "count_dont_know": 0
    }
    for survey in evaluated_surveys:
        if survey.sanety_check_passed:
            if evaluate_answer(survey.survey.granted_scenarios.aproximate_location, Answer.yes) and evaluate_answer(survey.survey.granted_scenarios.user_profiling, Answer.yes):
                result["count_true"] += 1
            elif evaluate_answer(survey.survey.granted_scenarios.aproximate_location, Answer.no) or evaluate_answer(survey.survey.granted_scenarios.user_profiling, Answer.no):
                result["count_false"] += 1
            elif evaluate_answer(survey.survey.granted_scenarios.aproximate_location, Answer.dont_know) or evaluate_answer(survey.survey.granted_scenarios.user_profiling, Answer.dont_know):
                result["count_dont_know"] += 1
            else:
                print("error neither true nor false nor don't know")
    return result

def evaluate_location_profiling_only_location(evaluated_surveys: List[EvaluatedSurvey]) -> Dict[str, int]:
    answers = [survey.survey.granted_scenarios.aproximate_location for survey in evaluated_surveys if survey.sanety_check_passed and evaluate_answer(survey.survey.granted_scenarios.user_profiling, Answer.dont_know)]
    return count_answers(answers, Answer.yes)

def evaluate_location_profiling_only_profiling(evaluated_surveys: List[EvaluatedSurvey]) -> Dict[str, int]:
    answers = [survey.survey.granted_scenarios.user_profiling for survey in evaluated_surveys if survey.sanety_check_passed and evaluate_answer(survey.survey.granted_scenarios.aproximate_location, Answer.dont_know)]
    return count_answers(answers, Answer.yes)

# 4) Devise Profiling (Q62, Q79)
#
def evaluate_device_profiling(evaluated_surveys: List[EvaluatedSurvey]) -> Dict[str, int]:
    result = {
        "count_true": 0,
        "count_false": 0,
        "count_dont_know": 0
    }
    for survey in evaluated_surveys:
        if survey.sanety_check_passed:
            if evaluate_answer(survey.survey.granted_scenarios.sensitive_device, Answer.yes) and evaluate_answer(survey.survey.granted_scenarios.detect_other_devices, Answer.yes):
                result["count_true"] += 1
            elif evaluate_answer(survey.survey.granted_scenarios.sensitive_device, Answer.no) or evaluate_answer(survey.survey.granted_scenarios.detect_other_devices, Answer.no):
                result["count_false"] += 1
            elif evaluate_answer(survey.survey.granted_scenarios.sensitive_device, Answer.dont_know) or evaluate_answer(survey.survey.granted_scenarios.detect_other_devices, Answer.dont_know):
                result["count_dont_know"] += 1
            else:
                print("error neither true nor false nor don't know")
                print(survey.survey.granted_scenarios.sensitive_device)
                print(survey.survey.granted_scenarios.exposing_devices)
    return result

def evaluate_at_least_on_threat(evaluated_surveys: List[EvaluatedSurvey]) -> Dict[str, int]:
    result = {"count_true": 0, "count_false": 0, "count_dont_know": 0}
    for survey in evaluated_surveys:
        if survey.sanety_check_passed:
            if (evaluate_answer(survey.survey.granted_scenarios.exposing_devices, Answer.yes) or 
                evaluate_answer(survey.survey.granted_scenarios.cross_user_tracking, Answer.yes) or 
                (evaluate_answer(survey.survey.granted_scenarios.aproximate_location, Answer.yes) and evaluate_answer(survey.survey.granted_scenarios.user_profiling, Answer.yes)) or 
                (evaluate_answer(survey.survey.granted_scenarios.sensitive_device, Answer.yes) and evaluate_answer(survey.survey.granted_scenarios.detect_other_devices, Answer.yes))):
                result["count_true"] += 1
            elif (evaluate_answer(survey.survey.granted_scenarios.exposing_devices, Answer.dont_know) and 
                evaluate_answer(survey.survey.granted_scenarios.cross_user_tracking, Answer.dont_know) and 
                (evaluate_answer(survey.survey.granted_scenarios.aproximate_location, Answer.dont_know) and evaluate_answer(survey.survey.granted_scenarios.user_profiling, Answer.dont_know)) and 
                (evaluate_answer(survey.survey.granted_scenarios.sensitive_device, Answer.dont_know) and evaluate_answer(survey.survey.granted_scenarios.detect_other_devices, Answer.dont_know))):
                result["count_dont_know"] += 1
            else:
                result["count_false"] += 1
    return result


def evaluate_device_profiling_stat_test(evaluated_surveys: List[EvaluatedSurvey]) -> Dict[str, int]:
    result = evaluate_device_profiling(evaluated_surveys)
    result["count_dont_know"] = result["count_dont_know"] + result["count_sensitive"]
    result["count_dont_know"] = result["count_dont_know"] + result["count_device"]
    del result["count_device"]
    del result["count_sensitive"]


    return result



def evaluate_device_profiling_only_sensitive(evaluated_surveys: List[EvaluatedSurvey]) -> Dict[str, int]:
    answers = [survey.survey.granted_scenarios.sensitive_device for survey in evaluated_surveys if survey.sanety_check_passed and evaluate_answer(survey.survey.granted_scenarios.detect_other_devices, Answer.dont_know)]
    return count_answers(answers, Answer.yes)

def evaluate_device_profiling_only_device(evaluated_surveys: List[EvaluatedSurvey]) -> Dict[str, int]:
    answers = [survey.survey.granted_scenarios.detect_other_devices for survey in evaluated_surveys if survey.sanety_check_passed and evaluate_answer(survey.survey.granted_scenarios.sensitive_device, Answer.dont_know)]
    return count_answers(answers, Answer.yes)


In [ ]:
#########################################
## Concepts                             #
#########################################
#

# 1) Proximity does not imply local network (Q65, Q66) 
#

def evaluate_proximity_v2(evaluated_surveys: List[EvaluatedSurvey]) -> Dict[str, str]:
    result = {
        "count_true": 0,
        "count_false": 0,
        "count_dont_know": 0,
    }
    for survey in evaluated_surveys:
        if survey.sanety_check_passed:
            if evaluate_answer(survey.survey.outside_scenarios.connect_smart_tv, Answer.no) and evaluate_answer(survey.survey.outside_scenarios.other_mobile_phones, Answer.no):
                result["count_true"] += 1
            elif evaluate_answer(survey.survey.outside_scenarios.connect_smart_tv, Answer.yes) or evaluate_answer(survey.survey.outside_scenarios.other_mobile_phones, Answer.yes):
                result["count_false"] += 1
            elif evaluate_answer(survey.survey.outside_scenarios.connect_smart_tv, Answer.dont_know) or evaluate_answer(survey.survey.outside_scenarios.other_mobile_phones, Answer.dont_know):
                result["count_dont_know"] += 1

    return result

def evaluate_proximity_v2_only_smart_tv(evaluated_surveys: List[EvaluatedSurvey]) -> Dict[str, str]:
    answers = [survey.survey.outside_scenarios.connect_smart_tv for survey in evaluated_surveys if survey.sanety_check_passed and evaluate_answer(survey.survey.outside_scenarios.other_mobile_phones, Answer.dont_know)]
    return count_answers(answers, Answer.no)

def evaluate_proximity_v2_only_mobile_phones(evaluated_surveys: List[EvaluatedSurvey]) -> Dict[str, str]:
    answers = [survey.survey.outside_scenarios.other_mobile_phones for survey in evaluated_surveys if survey.sanety_check_passed and evaluate_answer(survey.survey.outside_scenarios.connect_smart_tv, Answer.dont_know)]
    return count_answers(answers, Answer.no)

# 2) Boundries (Q67
#
# just count how many people get question right/wrong
def evaluate_boundries(evaluated_surveys: List[EvaluatedSurvey]) -> Dict[str, str]:

    answers = [survey.survey.outside_scenarios.connect_smart_tv_at_home for survey in evaluated_surveys if survey.sanety_check_passed]

    return count_answers(answers, Answer.no)


# 3) Transitivity, Taking the Permission with You (Q68, Q69)
#
# we use HomeScenario.smart_cast as the baseline
def evaluate_transitivity_1(evaluated_surveys: List[EvaluatedSurvey]) -> Dict[str, str]:
    result = { 
        "count_true": 0, # if baseline right cafe and work right
        "count_false": 0, # if baseline right cafe and work wrong
        "count_dont_know": 0
    }
    for survey in evaluated_surveys:
        if survey.sanety_check_passed:
            if HomeScenario.smart_cast not in survey.survey.at_home:
                continue
           
            if evaluate_answer(survey.survey.cafe_scenarios.connect_smart_tv, Answer.yes) and evaluate_answer(survey.survey.at_work_scenarios.connect_printer, Answer.yes):
                result["count_true"] += 1
            elif evaluate_answer(survey.survey.cafe_scenarios.connect_smart_tv, Answer.no) or evaluate_answer(survey.survey.at_work_scenarios.connect_printer, Answer.no):
                result["count_false"] += 1

            elif evaluate_answer(survey.survey.cafe_scenarios.connect_smart_tv, Answer.dont_know) or evaluate_answer(survey.survey.at_work_scenarios.connect_printer, Answer.dont_know):
                result["count_dont_know"] += 1
            else:
                print("error neither true nor false")
                print(survey.survey.cafe_scenarios.connect_smart_tv)
                print(survey.survey.at_work_scenarios.connect_printer)
    return result

def get_transitivity_1_false(evaluated_surveys: List[EvaluatedSurvey]) -> List[str]:
    result = []
    for survey in evaluated_surveys:
        if survey.sanety_check_passed:
            if HomeScenario.smart_cast not in survey.survey.at_home:
                continue
            
            if evaluate_answer(survey.survey.cafe_scenarios.connect_smart_tv, Answer.no) and evaluate_answer(survey.survey.at_work_scenarios.connect_printer, Answer.no):
                result.append(survey.survey.prolific_id)
    return result


def get_transitivity_1_base(evaluated_surveys: List[EvaluatedSurvey]) -> List[str]:
    result = {"count_true": 0, "count_false": 0, "count_dont_know": 0}
    for survey in evaluated_surveys:
        if survey.sanety_check_passed:
            if HomeScenario.dont_know in survey.survey.at_home:
                result["count_dont_know"] = result.get("count_dont_know", 0) + 1
            elif HomeScenario.smart_cast in survey.survey.at_home:
                result["count_true"] = result.get("count_true", 0) + 1
            elif HomeScenario.smart_cast not in survey.survey.at_home:
                result["count_false"] = result.get("count_false", 0) + 1
            
    return result

def evaluate_transitivity_1_only_cafe(evaluated_surveys: List[EvaluatedSurvey]) -> Dict[str, str]:
    answers = [survey.survey.cafe_scenarios.connect_smart_tv for survey in evaluated_surveys if survey.sanety_check_passed and evaluate_answer(survey.survey.at_work_scenarios.connect_printer, Answer.dont_know) and HomeScenario.discover_other_phones in survey.survey.at_home]
    return count_answers(answers, Answer.yes)

def evaluate_transitivity_1_only_work(evaluated_surveys: List[EvaluatedSurvey]) -> Dict[str, str]:
    answers = [survey.survey.at_work_scenarios.connect_printer for survey in evaluated_surveys if survey.sanety_check_passed and evaluate_answer(survey.survey.cafe_scenarios.connect_smart_tv, Answer.dont_know) and HomeScenario.discover_other_phones in survey.survey.at_home]
    return count_answers(answers, Answer.yes)


# 4) Transivity 2 (Q70, Q77) - Baseline HomeScenario.discover_other_phones
def get_transitivity_2_base(evaluated_surveys: List[EvaluatedSurvey]) -> List[str]:
    result = {"count_true": 0, "count_false": 0, "count_dont_know": 0}
    for survey in evaluated_surveys:
        if survey.sanety_check_passed:
            if HomeScenario.dont_know in survey.survey.at_home:
                result["count_dont_know"] = result.get("count_dont_know", 0) + 1
            elif HomeScenario.discover_other_phones in survey.survey.at_home:
                result["count_true"] = result.get("count_true", 0) + 1
            elif HomeScenario.discover_other_phones not in survey.survey.at_home:
                result["count_false"] = result.get("count_false", 0) + 1
            
    return result

def evaluate_transitivity_2(evaluated_surveys: List[EvaluatedSurvey]) -> Dict[str, str]:
    result = { 
        "count_true": 0, # if baseline right cafe and work right
        "count_false": 0, # if baseline right cafe and work wrong
        "count_dont_know": 0
    }
    for survey in evaluated_surveys:
        if survey.sanety_check_passed:
            if HomeScenario.discover_other_phones not in survey.survey.at_home:
                continue
            if evaluate_answer(survey.survey.cafe_scenarios.other_mobile_phones, Answer.yes) and evaluate_answer(survey.survey.at_work_scenarios.other_mobile_phones, Answer.yes):
                result["count_true"] += 1
            elif evaluate_answer(survey.survey.cafe_scenarios.other_mobile_phones, Answer.no) or evaluate_answer(survey.survey.at_work_scenarios.other_mobile_phones, Answer.no):
                result["count_false"] += 1
            elif evaluate_answer(survey.survey.cafe_scenarios.other_mobile_phones, Answer.dont_know) or evaluate_answer(survey.survey.at_work_scenarios.other_mobile_phones, Answer.dont_know):
                result["count_dont_know"] += 1
            else:
                print("error neither true nor false")
                print(survey.survey.cafe_scenarios.connect_smart_tv)
                print(survey.survey.at_work_scenarios.connect_printer)
    return result

def get_transitivity_2_false(evaluated_surveys: List[EvaluatedSurvey]) -> List[str]:
    result = []
    for survey in evaluated_surveys:
        if survey.sanety_check_passed:
            if HomeScenario.discover_other_phones not in survey.survey.at_home:
                continue
            if evaluate_answer(survey.survey.cafe_scenarios.other_mobile_phones, Answer.no) and evaluate_answer(survey.survey.at_work_scenarios.other_mobile_phones, Answer.no):
                result.append(survey.survey.prolific_id)
    return result

def evaluate_transitivity_2_only_cafe(evaluated_surveys: List[EvaluatedSurvey]) -> Dict[str, str]:
    answers = [survey.survey.cafe_scenarios.other_mobile_phones for survey in evaluated_surveys if survey.sanety_check_passed and evaluate_answer(survey.survey.at_work_scenarios.other_mobile_phones, Answer.dont_know) and HomeScenario.discover_other_phones  in survey.survey.at_home]
    return count_answers(answers, Answer.yes)

def evaluate_transitivity_2_only_work(evaluated_surveys: List[EvaluatedSurvey]) -> Dict[str, str]:
    answers = [survey.survey.at_work_scenarios.other_mobile_phones for survey in evaluated_surveys if survey.sanety_check_passed and evaluate_answer(survey.survey.cafe_scenarios.other_mobile_phones, Answer.dont_know) and HomeScenario.discover_other_phones  in survey.survey.at_home]
    return count_answers(answers, Answer.yes)




In [ ]:

#########################################
## Dummy Questions                      #
#########################################
#
# Ignore these questions: Q54, Q59, Q75


#########################################
## Misconceptions                       #
#########################################
#
# These are misconceptions we found online Q61
#

def evaluate_misconceptions(evaluated_surveys: List[EvaluatedSurvey]) ->  Dict[str, int]:
    result = {"wifi_password": 0,  "visible_phone": 0, "internet_access": 0, "bluetooth": 0, "any_misconception": 0}
    for survey in evaluated_surveys:
        if survey.sanety_check_passed:
            has_misconception = False
            if evaluate_answer(survey.survey.granted_scenarios.read_wifi_password, Answer.yes):
                result["wifi_password"] = result.get("wifi_password", 0)+ 1
                has_misconception = True
            if evaluate_answer(survey.survey.granted_scenarios.phone_become_visible, Answer.yes):
                result["visible_phone"] = result.get("visible_phone", 0)+ 1
                has_misconception = True
            if HomeScenario.internet_access in survey.survey.at_home:
                result["internet_access"] = result.get("internet_access", 0)+ 1
                has_misconception = True
            if HomeScenario.bluetooth in survey.survey.at_home:
                result["bluetooth"] = result.get("bluetooth", 0) + 1
                has_misconception = True
            if has_misconception:
                result["any_misconception"] = result.get("any_misconception", 0) + 1
    return result




In [ ]:
def get_valid_responses(evaluated_surveys: List[EvaluatedSurvey]) -> int:
    return len([survey for survey in evaluated_surveys if survey.sanety_check_passed])

def get_formatted_dict(result: Dict[str, int], total_number) -> Dict[str, str]:
    formatted_result = {}
    for key, value in result.items():
        formatted_result[key] = f"{value} ({value/total_number*100:.2f}%)"
    return formatted_result

In [ ]:
def get_local_network_score(evaluated_suverys: List[EvaluatedSurvey]) -> List[int]:
    result = []
    for survey in evaluated_suverys:
        if survey.sanety_check_passed:
            result.append(survey.ln_count_score if survey.ln_count_score >= 0 else 0)
    return result

def perform_p_test(evaluated_surveys: List[EvaluatedSurvey]):
    knows_local_network = []
    does_not_know_local_network = []
    for survey in evaluated_surveys:
        if survey.sanety_check_passed: #and survey.ln_count_score >= 0:
            score = survey.ln_count_score if survey.ln_count_score >= 0 else 0
            if survey.know_what_ln_is is True:
                knows_local_network.append(score)
                if survey.ln_count_score == -1:
                    print(survey.survey.prolific_id)
            elif survey.know_what_ln_is is False:
                does_not_know_local_network.append(score)
    print(statistics.mean(knows_local_network))
    print(statistics.mean(does_not_know_local_network))

    print(statistics.stdev(knows_local_network))
    print(statistics.stdev(does_not_know_local_network))

    print(statistics.median(knows_local_network))
    print(statistics.median(does_not_know_local_network))

    return ttest_ind(knows_local_network, does_not_know_local_network, alternative="greater")

In [ ]:
# function to calculate Cohen's d for independent samples
def cohend(d1, d2):
    # calculate the size of samples
    n1, n2 = len(d1), len(d2)
    # calculate the variance of the samples
    s1, s2 = var(d1, ddof=1), var(d2, ddof=1)
    # calculate the pooled standard deviation
    s = sqrt(((n1 - 1) * s1 + (n2 - 1) * s2) / (n1 + n2 - 2))
    # calculate the means of the samples
    u1, u2 = mean(d1), mean(d2)
    # calculate the effect size
    return (u1 - u2) / s

# Compute Cramér's V
def cramers_v(chi2, N, n, m): # Chi-square statistic, Total number of observations, Number of rows, Number of columns
    return sqrt(chi2 / (N * min(n-1, m-1)))


def chi_square_test(result_1, result_2):
    chi2, p, dof, expected = chi2_contingency([list(result_1.values()), list(result_2.values())])

    print("Chi-square statistic:", chi2)
    print("p-value:", p)    
    print("Degrees of freedom:", dof)
    print("Expected frequencies:")
    print(expected)
    # Calculate Cramér's V
    n = 0
    for x in list(result_1.values()) + list(result_2.values()):
        n = n + x
    print("phi", sqrt(chi2/n ))
    print(f"Cramér's V: {cramers_v(chi2, n, 2, len(result_1))}") # effect size
    return {"chi^2": chi2, "p-value": p, "dof": dof, "phi": sqrt(chi2/n ) }


In [ ]:
def get_results_rows_and_index(evaluated_surveys: List[EvaluatedSurvey], valid_responses):
    index = []
    results = []
    results.append(get_formatted_dict(evaluate_exposing_devices(evaluated_surveys), valid_responses))
    index.append("Exposing Devices")
    results.append(get_formatted_dict(evaluate_co_locating(evaluated_surveys), valid_responses))
    index.append("Co-locating")
    results.append(get_formatted_dict(evaluate_location_profiling(evaluated_surveys), valid_responses))
    index.append("Location Profiling")
    results.append(get_formatted_dict(evaluate_location_profiling_only_location(evaluated_surveys), valid_responses))
    index.append("Location Profiling-Only Location")
    results.append(get_formatted_dict(evaluate_location_profiling_only_profiling(evaluated_surveys), valid_responses))
    index.append("Location Profiling-Only Profiling")
    results.append(get_formatted_dict(evaluate_device_profiling(evaluated_surveys), valid_responses))
    index.append("Device Profiling")
    results.append(get_formatted_dict(evaluate_device_profiling_only_sensitive(evaluated_surveys), valid_responses))
    index.append("Device Profiling-Only Sensitive")
    results.append(get_formatted_dict(evaluate_device_profiling_only_device(evaluated_surveys), valid_responses))
    index.append("Device Profiling-Only Device")
    results.append(get_formatted_dict(evaluate_at_least_on_threat(evaluated_surveys), valid_responses))
    index.append("At Least One Threat")


    results.append(get_formatted_dict(evaluate_proximity_v2(evaluated_surveys), valid_responses))
    index.append("Proximity")
    results.append(get_formatted_dict(evaluate_proximity_v2_only_smart_tv(evaluated_surveys), valid_responses))
    index.append("Proximity-Only Smart TV")
    results.append(get_formatted_dict(evaluate_proximity_v2_only_mobile_phones(evaluated_surveys), valid_responses))
    index.append("Proximity-Only Mobile Phones")
    results.append(get_formatted_dict(evaluate_boundries(evaluated_surveys), valid_responses))
    index.append("Boundries")
    results.append(get_formatted_dict(get_transitivity_1_base(evaluated_surveys), valid_responses))
    index.append("Transitivity 1 Base")
    transitivity_1_count = 0
    for value in evaluate_transitivity_1(evaluated_surveys).values():
        transitivity_1_count += value
    results.append(get_formatted_dict(evaluate_transitivity_1(evaluated_surveys), transitivity_1_count ))
    index.append("Transitivity 1")
    results.append(get_formatted_dict(evaluate_transitivity_1_only_cafe(evaluated_surveys), transitivity_1_count))
    index.append("Transitivity 1-Only Cafe")
    results.append(get_formatted_dict(evaluate_transitivity_1_only_work(evaluated_surveys), transitivity_1_count))
    index.append("Transitivity 1-Only Work")
    results.append(get_formatted_dict(get_transitivity_2_base(evaluated_surveys), valid_responses))
    index.append("Transitivity 2 Base")
    transitivity_2_count = 0
    for value in evaluate_transitivity_1(evaluated_surveys).values():
        transitivity_2_count += value
    results.append(get_formatted_dict(evaluate_transitivity_2(evaluated_surveys), transitivity_2_count))
    index.append("Transitivity 2")
    results.append(get_formatted_dict(evaluate_transitivity_2_only_cafe(evaluated_surveys), transitivity_2_count))
    index.append("Transitivity 2-Only Cafe")
    results.append(get_formatted_dict(evaluate_transitivity_2_only_work(evaluated_surveys), transitivity_2_count))
    index.append("Transitivity 2-Only Work")

    results.append(get_formatted_dict(evaluate_misconceptions(evaluated_surveys), valid_responses))
    index.append("Misconceptions")

    return results, index


def get_result_rows_chi_square(know_ln_surveys: List[EvaluatedSurvey], do_not_know_ln_survey: List[EvaluatedSurvey]):
    results = []
    results.append(chi_square_test(evaluate_exposing_devices(know_ln_surveys), evaluate_exposing_devices(do_not_know_ln_survey)))
    results.append(chi_square_test(evaluate_co_locating(know_ln_surveys), evaluate_co_locating(do_not_know_ln_survey)))
    results.append(chi_square_test(evaluate_location_profiling(know_ln_surveys), evaluate_location_profiling(do_not_know_ln_survey)))
    results.append({})
    results.append({})
    results.append(chi_square_test(evaluate_device_profiling(know_ln_surveys), evaluate_device_profiling(do_not_know_ln_survey)))
    results.append(chi_square_test(evaluate_device_profiling_only_sensitive(know_ln_surveys), evaluate_device_profiling_only_sensitive(do_not_know_ln_survey)))
    results.append(chi_square_test(evaluate_device_profiling_only_device(know_ln_surveys), evaluate_device_profiling_only_device(do_not_know_ln_survey)))
    results.append(chi_square_test(evaluate_at_least_on_threat(know_ln_surveys), evaluate_at_least_on_threat(do_not_know_ln_survey)))

    results.append(chi_square_test(evaluate_proximity_v2(know_ln_surveys), evaluate_proximity_v2(do_not_know_ln_survey)))
    results.append({})
    results.append({})
    results.append(chi_square_test(evaluate_boundries(know_ln_surveys), evaluate_boundries(do_not_know_ln_survey)))
    results.append(chi_square_test(get_transitivity_1_base(know_ln_surveys), get_transitivity_1_base(do_not_know_ln_survey)))
    results.append(chi_square_test(evaluate_transitivity_1(know_ln_surveys), evaluate_transitivity_1(do_not_know_ln_survey)))
    results.append(chi_square_test(evaluate_transitivity_1_only_cafe(know_ln_surveys), evaluate_transitivity_1_only_cafe(do_not_know_ln_survey)))
    results.append(chi_square_test(evaluate_transitivity_1_only_work(know_ln_surveys), evaluate_transitivity_1_only_work(do_not_know_ln_survey)))
    results.append(chi_square_test(get_transitivity_2_base(know_ln_surveys), get_transitivity_2_base(do_not_know_ln_survey)))
    results.append(chi_square_test(evaluate_transitivity_2(know_ln_surveys), evaluate_transitivity_2(do_not_know_ln_survey)))
    results.append(chi_square_test(evaluate_transitivity_2_only_cafe(know_ln_surveys), evaluate_transitivity_2_only_cafe(do_not_know_ln_survey)))
    results.append(chi_square_test(evaluate_transitivity_2_only_work(know_ln_surveys), evaluate_transitivity_2_only_work(do_not_know_ln_survey)))
    results.append(chi_square_test(evaluate_misconceptions(know_ln_surveys), evaluate_misconceptions(do_not_know_ln_survey)))
    return results

def get_results_csv(evaluated_suverys: List[EvaluatedSurvey], valid_responses, file_name: str):
    results, index = get_results_rows_and_index(evaluated_suverys, valid_responses)
    df = pd.DataFrame(results, index=index)
    df.to_csv(file_name)


def append_to_keys(result, append):
    to_return = {}
    for key, value in result.items():
        to_return[key + append] = value
    return to_return

def append_all_to_keys(result, append):
    to_return = []
    for r in result:
        to_return.append(append_to_keys(r, append))
    return to_return

def get_results_csv_with_x_squared(know_ln_surveys: List[EvaluatedSurvey], do_not_know_ln_survey: List[EvaluatedSurvey], file_name: str):
    results_ln, index = get_results_rows_and_index(know_ln_surveys, len(know_ln_surveys))
    results_nln, _ = get_results_rows_and_index(do_not_know_ln_survey, len(do_not_know_ln_survey))
    results_ln = append_all_to_keys(results_ln, "_know_ln")
    results_nln = append_all_to_keys(results_nln, "_do_not_know")
    results_chi_square = get_result_rows_chi_square(know_ln_surveys, do_not_know_ln_survey)
    df_1 = pd.DataFrame(results_ln, index=index)
    df_2 = pd.DataFrame(results_nln, index=index)
    df_3 = pd.DataFrame(results_chi_square, index=index)
    df = pd.concat([df_1, df_2, df_3], axis=1)
    df.to_csv(file_name)
    

In [ ]:


survey_results = parse_survey(survey_file)

In [ ]:
manual_decision = parse_manual_file(manual_ln_label_file)

In [ ]:
evaluated_suverys = evaluate_surveys(survey_results)

In [ ]:
update_local_network_knowledge_from_manual_analysis(evaluated_suverys, manual_decision)

In [ ]:
evaluated_suverys_know_what_ln_is = [survey for survey in evaluated_suverys if survey.know_what_ln_is is True and survey.sanety_check_passed]
evaluated_suverys_do_not_know_what_ln_is = [survey for survey in evaluated_suverys if survey.know_what_ln_is is False and survey.sanety_check_passed]
evaluated_suverys_techical_background = [survey for survey in evaluated_suverys if survey.has_it_background is True and survey.sanety_check_passed]
evaluated_suverys_no_techical_background = [survey for survey in evaluated_suverys if survey.has_it_background is False and survey.sanety_check_passed]

In [ ]:
c = 0
for e in evaluated_suverys:
    if e.sanety_check_passed and not e.know_what_ln_is:
        c += 1
print(c)

In [ ]:
perform_p_test(evaluated_suverys)

In [ ]:
cohend(get_local_network_score(evaluated_suverys_know_what_ln_is), get_local_network_score(evaluated_suverys_do_not_know_what_ln_is))

In [ ]:
valid_responses = get_valid_responses(evaluated_suverys)

In [ ]:
get_survey_time(evaluated_suverys)

In [ ]:
get_ati_total_score(evaluated_suverys)

In [ ]:
get_percentage_demographic(evaluated_suverys, number_of_results=None)

In [ ]:
get_formatted_dict(count_permission_seen(evaluated_suverys), valid_responses)

In [ ]:
len(evaluated_suverys_techical_background)/valid_responses*100

In [ ]:
#########################################
## Use Cases                            #
#########################################
# 
# Evaluate use cases from Q3.3
# 

print(get_formatted_dict(evaluate_use_cases(evaluated_suverys), valid_responses))

In [ ]:
#########################################
## Attacker Models                      #
#########################################
# 
# 1) Threat of exposing offline devices (Q60)
#

# 
print("exposing devices")
print(get_formatted_dict(evaluate_exposing_devices(evaluated_suverys), valid_responses))

# 2) People co-locating (Q58)
#

# how many are aware of this threat.
#
print("People co-locating")
print(get_formatted_dict(evaluate_co_locating(evaluated_suverys), valid_responses))

# 3) Location Profiling (Q56, Q57)
#

#

print("location")
print(get_formatted_dict(evaluate_location_profiling(evaluated_suverys), valid_responses))
print(get_formatted_dict(evaluate_location_profiling_only_location(evaluated_suverys), valid_responses))
print(get_formatted_dict(evaluate_location_profiling_only_profiling(evaluated_suverys), valid_responses))

# 4) Devise Profiling (Q62, Q79)
#



print("device")
print(get_formatted_dict(evaluate_device_profiling(evaluated_suverys), valid_responses))
print(get_formatted_dict(evaluate_device_profiling_only_sensitive(evaluated_suverys), valid_responses))
print(get_formatted_dict(evaluate_device_profiling_only_device(evaluated_suverys), valid_responses))




# At least one threat
print("at least on threat")
print(get_formatted_dict(evaluate_at_least_on_threat(evaluated_suverys), valid_responses))

In [ ]:

#########################################
## Concepts                             #
#########################################
#
# 1) Proximity does not imply local network (Q65, Q66) 
#

print("proximity")
print(get_formatted_dict(evaluate_proximity_v2(evaluated_suverys), valid_responses))
print(get_formatted_dict(evaluate_proximity_v2_only_smart_tv(evaluated_suverys), valid_responses))
print(get_formatted_dict(evaluate_proximity_v2_only_mobile_phones(evaluated_suverys), valid_responses))

# 2) Boundries (Q67
#

print("boundries")
print(get_formatted_dict(evaluate_boundries(evaluated_suverys), valid_responses))

# 3) Transitivity, Taking the Permission with You (Q68, Q69)
#
print("transitivity - 1")
print(get_formatted_dict(get_transitivity_1_base(evaluated_suverys), valid_responses))
print(get_formatted_dict(evaluate_transitivity_1(evaluated_suverys), valid_responses))
print(get_formatted_dict(evaluate_transitivity_1_only_cafe(evaluated_suverys), valid_responses))
print(get_formatted_dict(evaluate_transitivity_1_only_work(evaluated_suverys), valid_responses))


# 4) Transitivity 2 - (Q70, Q77)
print("transitivity - 2")
print(get_formatted_dict(get_transitivity_2_base(evaluated_suverys), valid_responses))
print(get_formatted_dict(evaluate_transitivity_2(evaluated_suverys), valid_responses))
print(get_formatted_dict(evaluate_transitivity_2_only_cafe(evaluated_suverys), valid_responses))
print(get_formatted_dict(evaluate_transitivity_2_only_work(evaluated_suverys), valid_responses))



In [ ]:
len(set(get_transitivity_1_false(evaluated_suverys)).intersection(get_transitivity_2_false(evaluated_suverys)))

In [ ]:


#########################################
## Misconceptions                       #
#########################################
#
# These are misconceptions we found online Q61
#

print(get_formatted_dict(evaluate_misconceptions(evaluated_suverys), valid_responses))


In [ ]:
# If tech background better local nework understanding?
print(get_formatted_dict(count_ln_knowledge(evaluated_suverys_techical_background), len(evaluated_suverys_techical_background)))
print(get_formatted_dict(count_ln_knowledge(evaluated_suverys_no_techical_background), len(evaluated_suverys_no_techical_background)))
chi_square_test(count_ln_knowledge(evaluated_suverys_techical_background), count_ln_knowledge(evaluated_suverys_no_techical_background))


In [ ]:
# If ln understanding - better exposing devices?

print(get_formatted_dict(evaluate_exposing_devices(evaluated_suverys_know_what_ln_is), len(evaluated_suverys_know_what_ln_is)))
print(get_formatted_dict(evaluate_exposing_devices(evaluated_suverys_do_not_know_what_ln_is), len(evaluated_suverys_do_not_know_what_ln_is)))
chi_square_test(evaluate_exposing_devices(evaluated_suverys_know_what_ln_is), evaluate_exposing_devices(evaluated_suverys_do_not_know_what_ln_is))


In [ ]:
# If ln understanding - better device profiling?

print(get_formatted_dict(evaluate_device_profiling(evaluated_suverys_know_what_ln_is), len(evaluated_suverys_know_what_ln_is)))
print(get_formatted_dict(evaluate_device_profiling(evaluated_suverys_do_not_know_what_ln_is), len(evaluated_suverys_do_not_know_what_ln_is)))
chi_square_test(evaluate_device_profiling(evaluated_suverys_know_what_ln_is), evaluate_device_profiling(evaluated_suverys_do_not_know_what_ln_is))

In [ ]:
# If ln understanding - better co locationg understanding?

print(get_formatted_dict(evaluate_co_locating(evaluated_suverys_know_what_ln_is), len(evaluated_suverys_know_what_ln_is)))
print(get_formatted_dict(evaluate_co_locating(evaluated_suverys_do_not_know_what_ln_is), len(evaluated_suverys_do_not_know_what_ln_is)))
chi_square_test(evaluate_co_locating(evaluated_suverys_know_what_ln_is), evaluate_co_locating(evaluated_suverys_do_not_know_what_ln_is))

In [ ]:
# If ln understanding - better location profiling understanding?

print(get_formatted_dict(evaluate_location_profiling(evaluated_suverys_know_what_ln_is), len(evaluated_suverys_know_what_ln_is)))
print(get_formatted_dict(evaluate_location_profiling(evaluated_suverys_do_not_know_what_ln_is), len(evaluated_suverys_do_not_know_what_ln_is)))

chi_square_test(evaluate_location_profiling(evaluated_suverys_know_what_ln_is), evaluate_location_profiling(evaluated_suverys_do_not_know_what_ln_is))

In [ ]:
# If ln understanding - are aware of at least one threat
print(get_formatted_dict(evaluate_at_least_on_threat(evaluated_suverys_know_what_ln_is), len(evaluated_suverys_know_what_ln_is)))
print(get_formatted_dict(evaluate_at_least_on_threat(evaluated_suverys_do_not_know_what_ln_is), len(evaluated_suverys_do_not_know_what_ln_is)))

chi_square_test(evaluate_at_least_on_threat(evaluated_suverys_know_what_ln_is), evaluate_at_least_on_threat(evaluated_suverys_do_not_know_what_ln_is))

In [ ]:
# If ln understanding - are aware of proximity

print(get_formatted_dict(evaluate_proximity_v2(evaluated_suverys_know_what_ln_is), len(evaluated_suverys_know_what_ln_is)))
print(get_formatted_dict(evaluate_proximity_v2(evaluated_suverys_do_not_know_what_ln_is), len(evaluated_suverys_do_not_know_what_ln_is)))

chi_square_test(evaluate_proximity_v2(evaluated_suverys_know_what_ln_is), evaluate_proximity_v2(evaluated_suverys_do_not_know_what_ln_is))


In [ ]:
# If ln understanding - are aware of network boundaries

print(get_formatted_dict(evaluate_boundries(evaluated_suverys_know_what_ln_is), len(evaluated_suverys_know_what_ln_is)))
print(get_formatted_dict(evaluate_boundries(evaluated_suverys_do_not_know_what_ln_is), len(evaluated_suverys_do_not_know_what_ln_is)))
chi_square_test(evaluate_boundries(evaluated_suverys_know_what_ln_is), evaluate_boundries(evaluated_suverys_do_not_know_what_ln_is))



In [ ]:
# If ln understanding - are aware of transivity

print(get_formatted_dict(get_transitivity_1_base(evaluated_suverys_know_what_ln_is), len(evaluated_suverys_know_what_ln_is)))
print(get_formatted_dict(get_transitivity_1_base(evaluated_suverys_do_not_know_what_ln_is), len(evaluated_suverys_do_not_know_what_ln_is)))
chi_square_test(get_transitivity_1_base(evaluated_suverys_know_what_ln_is), get_transitivity_1_base(evaluated_suverys_do_not_know_what_ln_is))
print("-------")


eval1_ln = evaluate_transitivity_1(evaluated_suverys_know_what_ln_is)
eval1_nln = evaluate_transitivity_1(evaluated_suverys_do_not_know_what_ln_is)

print(get_formatted_dict(eval1_ln, len(evaluated_suverys_know_what_ln_is)))
print(get_formatted_dict(eval1_nln, len(evaluated_suverys_do_not_know_what_ln_is)))

chi_square_test(eval1_ln, eval1_nln)
print("-------")



# If ln understanding - are aware of transivity 2

print("transitivity_2 - base")
print(get_formatted_dict(get_transitivity_2_base(evaluated_suverys_know_what_ln_is), len(evaluated_suverys_know_what_ln_is)))
print(get_formatted_dict(get_transitivity_2_base(evaluated_suverys_do_not_know_what_ln_is), len(evaluated_suverys_do_not_know_what_ln_is)))
chi_square_test(get_transitivity_2_base(evaluated_suverys_know_what_ln_is), get_transitivity_2_base(evaluated_suverys_do_not_know_what_ln_is))
print("-------")

eval2_ln = evaluate_transitivity_2(evaluated_suverys_know_what_ln_is)
eval2_nln = evaluate_transitivity_2(evaluated_suverys_do_not_know_what_ln_is)

print(get_formatted_dict(eval2_ln, 15+ 29 + 16)) # right base question ln 
print(get_formatted_dict(eval2_nln, 11 + 26 + 12)) # right base question nln

chi_square_test(eval2_ln, eval2_nln)
print("-------")

In [ ]:
# If ln understanding - are aware of misconceptions

print(get_formatted_dict(evaluate_misconceptions(evaluated_suverys_know_what_ln_is), len(evaluated_suverys_know_what_ln_is)))

print(get_formatted_dict(evaluate_misconceptions(evaluated_suverys_do_not_know_what_ln_is), len(evaluated_suverys_do_not_know_what_ln_is)))
chi_square_test(evaluate_misconceptions(evaluated_suverys_know_what_ln_is), evaluate_misconceptions(evaluated_suverys_do_not_know_what_ln_is))


In [ ]:
get_results_csv(evaluated_suverys, valid_responses, "evaluate_survey_all.csv")

In [ ]:
get_results_csv_with_x_squared(evaluated_suverys_know_what_ln_is, evaluated_suverys_do_not_know_what_ln_is, "evaluate_survey_compare.csv")